In [44]:
# Import qiskit and modules
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import time

from math import pi

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile, assemble
from qiskit import Aer
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
from qiskit.circuit.library import TwoLocal

from scipy.optimize import minimize

from AtomLoader import *

In [45]:
################ Tutorial Block ################
# Initializing a three-qubit quantum state
desired_vector = [
    math.cos(np.pi/4),
    0,
    math.cos(np.pi/4),
    0,
    0,
    0,
    0,
    0]


q = QuantumRegister(3)

qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1],q[2]])
qc.draw()

backend = BasicAer.get_backend('statevector_simulator')
job = backend.run(transpile(qc, backend))
qc_state = job.result().get_statevector(qc)
qc_state

state_fidelity(desired_vector,qc_state)

1.0

In [46]:
def training_circuit(parameters, atomic_descriptors):
    
    qc = QuantumCircuit(M)
    qc_descriptor = QuantumCircuit(M)

    list_ksi = parameters ## twolocal have (depth+1)*M*2 parameters
    
    # for i in range(M):
    for i, descriptor in enumerate(atomic_descriptors):
        qc_descriptor.u(
            descriptor[0],
            descriptor[1],
            0,
            i
            ) #should be replaced to theta(list_eta[i]), phi(list_ksi[i])
        #Young Oh should read this!

    twolocal = TwoLocal(num_qubits=M, reps=depth, rotation_blocks=['ry','rz'], 
                   entanglement_blocks='cx', entanglement='circular', parameter_prefix='ξ', insert_barriers=True)
    
    twolocal = twolocal.bind_parameters(list_ksi)
    
    qc += qc_descriptor
    qc.barrier()

    qc += twolocal
    qc.barrier()

    #Observable
    qc.z(0)

    qc.barrier()

    qc += twolocal.inverse()
    qc.barrier()

    qc += qc_descriptor.inverse()
    qc.barrier()
    
    return qc

def calculate_loss_function(parameters, list_eta, molecule_index):

    backend = Aer.get_backend('aer_simulator')

    energy = 0
    #list_eta = parameters[0:M]
    atom_data = AtomLoader(list_eta, idx=[molecule_index])
    descriptors = atom_data[molecule_index]['descriptor']
    n_atoms = len(atom_data[molecule_index]['descriptor'])
    ground_energy_label = atom_data[molecule_index]['ground_energy'][0] / -20000 * n_atoms

    for i, atomic_descriptors in enumerate(descriptors):

        qctl = QuantumRegister(M)
        qc = ClassicalRegister(M)
        circ = QuantumCircuit(qctl, qc)

        circ += training_circuit(parameters, atomic_descriptors)

        circ.save_statevector()
        t_circ = transpile(circ, backend)
        qobj = assemble(t_circ)
        job = backend.run(qobj)

        result = job.result()
        outputstate = result.get_statevector(circ, decimals=100)
        o = outputstate

        energy += np.abs(o[0])+np.abs([2**(M-1)]) ## <0|GdWdOWG|0> is picking first component of GdWdOWG|0>.
    
    
    loss = ((energy - ground_energy_label)/n_atoms)**2

    return loss

In [51]:
M = 4 #Number of descriptors (features) = Number of qubits
#N = 3 #Number of atoms = Number of circuits
depth = 2 #Number of repetitions of layer
len_param = (depth+1)*M*2 #Number of total parameters (eta + ksi)

print("Number of total parameters:", len_param)
print()

random_indexes = [str(x) for x in (np.random.choice(len(qm9), 1) + 1)]

x0=[float(random.randint(0,3000))/1000 for i in range(0, len_param)]
list_eta = np.array(range(0,M))/(2*M)
print(list_eta)

start = time.time()

for i,molecule_index in enumerate(random_indexes):

    print("Current index of molecule:", i+1)
    
    print(calculate_loss_function(x0,list_eta,molecule_index))
    
    out = minimize(calculate_loss_function,
    x0,
    method='L-BFGS-B',
    options={'maxiter':300},
    args=(list_eta, molecule_index)
    )
    
    x0 = out.x
    print("Current parameters: ", x0)
    
    print("Current Loss Function:",out['fun'])
    print()

end = time.time()

print(f"Calculation time: {end-start}s\n")

print(out)

print(out['x'])

Number of total parameters: 24

[0.    0.125 0.25  0.375]
Current index of molecule: 1
[56.58834541]
Current parameters:  [ 2.81535331  0.44712249  1.18016131  0.50150048 -0.00558005  1.02432623
  2.32105198  0.20523821  1.54752961  0.27545953  1.86500067  0.00919275
  1.58423185  1.96022262  0.264       1.61699894  0.08666054  2.92799894
  2.242       2.34400067  0.36799989  0.56499894  2.64300067  2.59399894]
Current Loss Function: [55.59489075]

Calculation time: 3556.7498621940613s

      fun: array([55.59489075])
 hess_inv: <24x24 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 4.59081664e-02, -1.99846539e-01, -1.15779387e-01,  5.34328135e-04,
       -3.57322705e-01,  1.00909148e-01, -1.14987132e-02,  5.15854026e-04,
        7.70590707e-02, -1.54379620e-02,  0.00000000e+00,  3.54537377e-01,
        8.91319024e-02, -4.73718845e-03,  0.00000000e+00,  0.00000000e+00,
        2.39428033e-01,  0.00000000e+00,  2.84217096e-06,  0.00000000e+00,
        0.00000000e+00,  0.00000